In [2]:
import os
import numpy as np
from tensorflow.keras.models import load_model
import math
import time
import threading
import cv2

from playsound import playsound

# mengambil aturan Haar-cascade
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt.xml"
cascRPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_righteye_2splits.xml"
cascLPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_lefteye_2splits.xml"

def prepare(citra):
    IMG_SIZE = 48
    resized_array = cv2.resize(citra, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

faceCascade = cv2.CascadeClassifier(cascPathface)
eyeLCascade = cv2.CascadeClassifier(cascLPatheyes)
eyeRCascade = cv2.CascadeClassifier(cascRPatheyes)

video_capture = cv2.VideoCapture(0)

# model = load_model("./drowsiness_vgg_non_gpu_wino.h5", custom_objects=custom_objects)
model = load_model("./drowsiness_vgg_non_gpu_50.h5")

# used to record the time when we processed last frame
fps_start_time = time.time()
start_time_avg_fps = time.time()
detected_time = time.time()

detected = False
start_time = time.time()
# used to record the time at which we processed current frame
fps = 0
total_fps = 0
average_fps = 0
frame_count = 0
counter = 0

class_r = 0
class_l = 0
class_mouth = 0

sound_playing = 0

def play_alarm_sound():
    global sound_playing
    sound_playing = 1
    playsound('alarm.mp3')
    sound_playing = 0 # Jeda 2 detik setelah memutar suara

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    height,width = frame.shape[:2]
    memulai = time.time()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    left_eye = eyeLCascade.detectMultiScale(gray)
    right_eye =  eyeRCascade.detectMultiScale(gray)
    mouth = faceCascade.detectMultiScale(gray)
    
    show_class_l = ""
    show_class_r = ""
    
    fps_end_time = time.time()
    fps = round(1 / (fps_end_time - fps_start_time))
    fps_start_time = fps_end_time
    total_fps += fps
    frame_count += 1
    
    if fps_end_time - start_time_avg_fps >= 1:
        average_fps = round(total_fps / frame_count)

        frame_count = 0
        total_fps = 0
        start_time_avg_fps = fps_end_time
    
    
    # ekstraksi wajah dan mata
    for (x,y,w,h) in right_eye:
        r_eye = frame[y:y+h,x:x+w]
        rpred = model.predict(prepare(r_eye))
        class_r = np.argmax(rpred,axis=1)
        
    for (x,y,w,h) in left_eye:
        l_eye= frame[y:y+h,x:x+w]
        lpred = model.predict(prepare(l_eye))
        class_l = np.argmax(lpred,axis=1)
        
    if class_l == 2:
        show_class_l = "Mata kiri tertutup"
    else:
        show_class_l = "Mata kiri terbuka"
    
    if class_r == 2:
        show_class_r = "Mata kanan tertutup"
    else:
        show_class_r = "Mata kanan terbuka"
        
    if class_mouth == 1:
        show_mouth = "Wajah menguap"
    else:
        show_mouth = "Wajah tidak menguap"
        
    for (x,y,w,h) in mouth:
        mouth_frame = frame[y:y+h,x:x+w]
        mouthpred = model.predict(prepare(mouth_frame))
        class_mouth = np.argmax(mouthpred,axis=1)
        if detected_time >= 3:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 5)
            cv2.putText(frame, 'Drowsy', (x, (y - 10)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(frame, f'{str(show_class_l)}', ((x - 220), (y + 30)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8,(255,255,255))
            cv2.putText(frame, f'{str(show_class_r)}', ((x - 220), (y + 70)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8,(255,255,255))
            cv2.putText(frame, f'{str(show_mouth)}', ((x - 220), (y + 110)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8,(255,255,255))
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 5)
            cv2.putText(frame, 'Steady', (x, (y - 10)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(frame, f'{str(show_class_l)}', ((x - 220), (y + 30)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8,(255,255,255))
            cv2.putText(frame, f'{str(show_class_r)}', ((x - 220), (y + 70)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8,(255,255,255))
            cv2.putText(frame, f'{str(show_mouth)}', ((x - 220), (y + 110)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8,(255,255,255))
                        
    current_time = time.time() - start_time
    # estimasi waktu
    if(class_mouth == 1 or (class_r == 2 and class_l == 2)):
        if not detected:
            start_time_detected = time.time()  # Catat waktu mulai jika kondisi baru terpenuhi
        detected = True
        detected_time = round(time.time() - start_time_detected)
    else:
        detected = False
        detected_time = 0

    cv2.putText(frame,'FPS: '+str(f'{fps : .2f}'),(350,height-425), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(frame,'Average FPS: '+str(f'{average_fps : .2f}'),(350,height-375), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(frame,'Time: '+str(f'{current_time : .2f}'),(350,height-325), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(frame,'Score: '+str(f'{detected_time}'),(width // 2,height - 100), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    
    # kondisi pengemudi
    if(detected_time >= 3 and not sound_playing):
        alarm_thread = threading.Thread(target=play_alarm_sound)
        alarm_thread.start()# Ganti dengan path file suara alarm Anda
    
    akhir = time.time()
    delay = akhir - memulai
    
    print(delay)
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
# save_path = "./ss_program/Uji_VGG100/frame_terakhir.png"
# file_name, file_extension = os.path.splitext(save_path)
# counter_image = 1
# while os.path.exists(save_path):
#     save_path = f"{file_name}({counter_image}){file_extension}"
#     counter_image += 1
# cv2.imwrite(save_path, frame)

video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 32ms/step
0.7674221992492676
1/1 [==============================] - 0s 51ms/step
0.27922844886779785
1/1 [==============================] - 0s 35ms/step
0.31802940368652344
1/1 [==============================] - 0s 41ms/step
0.712024450302124
1/1 [==============================] - 0s 30ms/step
0.655386209487915
1/1 [==============================] - 0s 21ms/step
0.5745627880096436
1/1 [==============================] - 0s 22ms/step
0.3629496097564697
1/1 [==============================] - 0s 23ms/step
0.29320263862609863
1/1 [==============================] - 0s 24ms/step
0.2831392288208008
1/1 [==============================] - 0s 23ms/step
0.28832030296325684
1/1 [==============================] - 0s 23ms/step
0.2860891819000244
1/1 [==============================] - 0s 23ms/step
0.2543678283691406
1/1 [==============================] - 0s 23ms/step
0.24640250205993652
1/1 [==============================] - 0s 22ms/step
0.25656962394714355
1/

In [1]:
import os
import numpy as np
import tensorflow as tf
import math
import time
import threading
import cv2
import pygame

# mengambil aturan Haar-cascade
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt.xml"
cascRPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_righteye_2splits.xml"
cascLPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_lefteye_2splits.xml"


faceCascade = cv2.CascadeClassifier(cascPathface)
eyeLCascade = cv2.CascadeClassifier(cascRPatheyes)
eyeRCascade = cv2.CascadeClassifier(cascLPatheyes)

video_capture = cv2.VideoCapture(0)

interpreter = tf.lite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# used to record the time when we processed last frame
fps_start_time = time.time()
  
# used to record the time at which we processed current frame
fps = 0

start_time = time.time()
total_duration = 0
detected_time = time.time()

class_mouth = 0
r_class_eye = 0
l_class_eye = 0

sound_playing = 0

detected = False

def play_alarm_sound():
    global sound_playing
    sound_playing = 1
    playsound('alarm.mp3')
    sound_playing = 0 # Jeda 2 detik setelah memutar suara

while True:
    ret, frame = video_capture.read()
    height,width = frame.shape[:2]
    
    memulai = time.time()
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    l_eye = eyeLCascade.detectMultiScale(gray)
    r_eye = eyeRCascade.detectMultiScale(gray)
    mouth = faceCascade.detectMultiScale(gray, minSize=(180, 180))
    
    fps_end_time = time.time()
    fps = round(1 / (fps_end_time - fps_start_time))
    fps_start_time = fps_end_time
    
    if l_class_eye == 2:
        l_show = "Mata kiri tertutup"
    else:
        l_show = "Mata kiri terbuka"
    
    if r_class_eye == 2:
        r_show = "Mata kanan tertutup"
    else:
        r_show = "Mata kanan terbuka"
    
    if class_mouth == 1:
        show_mouth = "Wajah menguap"
    else:
        show_mouth = "Wajah tidak menguap"
    
    for (x,y,w,h) in l_eye:
        l_eye_frame = frame[y:y+h,x:x+w]
        l_eye_frame_resize = np.resize(l_eye_frame, (1, 48, 48, 3)).astype(np.float32)
        # Set input tensor ke dalam interpreter
        interpreter.set_tensor(input_details[0]['index'], l_eye_frame_resize)

        # Jalankan inferensi
        interpreter.invoke()

        # Dapatkan output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])


        # Mengambil label kelas yang memiliki probabilitas tertinggi
        l_class_eye = np.argmax(output_data)
    
    for (x,y,w,h) in r_eye:
        r_eye_frame = frame[y:y+h,x:x+w]
        r_eye_frame_resize = np.resize(r_eye_frame, (1, 48, 48, 3)).astype(np.float32)
        # Set input tensor ke dalam interpreter
        interpreter.set_tensor(input_details[0]['index'], r_eye_frame_resize)

        # Jalankan inferensi
        interpreter.invoke()

        # Dapatkan output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])


        # Mengambil label kelas yang memiliki probabilitas tertinggi
        r_class_eye = np.argmax(output_data)
        
    for (x,y,w,h) in mouth:
        mouth_frame = frame[y:y+h,x:x+w]
        mouth_frame_resize = np.resize(mouth_frame, (1, 48, 48, 3)).astype(np.float32)
        # Set input tensor ke dalam interpreter
        interpreter.set_tensor(input_details[0]['index'], mouth_frame_resize)

        # Jalankan inferensi
        interpreter.invoke()

        # Dapatkan output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])


        # Mengambil label kelas yang memiliki probabilitas tertinggi
        class_mouth = np.argmax(output_data)
        if detected_time >= 3:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 5)
            cv2.putText(frame, 'Drowsy', (x, (y - 10)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(frame, l_show, ((x - 300), (y + 30)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255))
            cv2.putText(frame, r_show, ((x - 300), (y + 70)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255))
            cv2.putText(frame, show_mouth, ((x - 300), (y + 110)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255))
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 5)
            cv2.putText(frame, 'Steady', (x, (y - 10)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(frame, l_show, ((x - 300), (y + 30)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255))
            cv2.putText(frame, r_show, ((x - 300), (y + 70)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255))
            cv2.putText(frame, show_mouth, ((x - 300), (y + 110)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255))                        
    
    # estimasi waktu
    #if(class_mouth == 1):
    if(class_mouth == 1 or (r_class_eye == 2 and l_class_eye == 2)):
        if not detected:
            start_time_detected = time.time()  # Catat waktu mulai jika kondisi baru terpenuhi
        detected = True
        detected_time = round(time.time() - start_time_detected)
    else:
        detected = False
        detected_time = 0
        
    cv2.putText(frame,'Terdeteksi:'+str(f'{detected_time}'),(380,height-100), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(frame,'FPS:'+str(f'{fps : .2f}'),(380,height-300), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    #cv2.putText(frame,'FPS rata-rata:'+str(f'{class_mouth : .2f}'),(380,height-400), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)

    # kondisi pengemudi
    if(detected_time >= 3 and not sound_playing):
        alarm_thread = threading.Thread(target=play_alarm_sound)
        alarm_thread.start()# Ganti dengan path file suara alarm Anda
        
    akhir = time.time()
    delay = akhir - memulai
    
    cv2.putText(frame,'Delay:'+str(f'{delay : .2f}'),(380,height-275), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()



C:\Users\Lenovo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
